# Imports

Started up by running `shawnhammer` in a terminal. Running on slot 4, /home/cryo/docker/pysmurf/dev/v5.0.2, rogue version /home/cryo/docker/smurf/stable/slotN/v5.0.2, and experiment cfg cfg_files/b33/experiment_b33_lbhb.cfg. pysmurf branch b33_setup&issue658.  `shawnhammer` already runs `setup()`, so no need to run here.

In [1]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import yaml
import time
import pickle as pkl
import os
import numpy as np
from matplotlib import cm
from scipy.signal import welch,periodogram
import pandas as pd
import glob
from scipy.optimize import curve_fit

import pysmurf.client
import matplotlib.pylab as plt
import numpy as np
import sys
import os
epics_prefix="smurf_server_s4"
config_file=os.path.abspath("/usr/local/src/pysmurf/cfg_files/b33/experiment_b33_lbhb.cfg")
S = pysmurf.client.SmurfControl(epics_root=epics_prefix,cfg_file=config_file,setup=False,make_logfile=False,shelf_manager="shm-smrf-sp01")

Try to fill bands.  Running fill_band.py in my scratch with these settings:

```
amplitude=12
freq_std_khz=100
freq_std_mhz=freq_std_khz/1000.
one_subband_at_a_time=False
wait_btw_subbands_sec=0.1

restrict_nper_band=None
restrict_nper_band=250
```

That should give 250 tones in each band I run it on.


In [8]:
for band in range(8):
    sys.argv[1]=band
    exec(open('/usr/local/src/pysmurf/scratch/shawn/fill_band.py').read())

filling bands 0
-> Restricting nchan to 250.
filling bands 1
-> Restricting nchan to 250.
filling bands 2
-> Restricting nchan to 250.
filling bands 3
-> Restricting nchan to 250.
filling bands 4
-> Restricting nchan to 250.
filling bands 5
-> Restricting nchan to 250.
filling bands 6
-> Restricting nchan to 250.
filling bands 7
-> Restricting nchan to 250.


Here's what that looks like on signal analyzer picked off using a 16 dB coupler (and maybe 4ft cables from SMuRF to in/out of directional coupler, and from directional coupler coupled port to signal analyzer:

In [12]:
for band in range(8):
    print(f'S.get_synthesis_scale({band})={S.get_synthesis_scale(band)}')
    print(f'S.get_att_uc({band})={S.get_att_uc(band)}')
    print(f'S.get_att_dc({band})={S.get_att_dc(band)}')
    print('--------')

S.get_synthesis_scale(0)=1
S.get_att_uc(0)=0
S.get_att_dc(0)=0
--------
S.get_synthesis_scale(1)=1
S.get_att_uc(1)=0
S.get_att_dc(1)=0
--------
S.get_synthesis_scale(2)=1
S.get_att_uc(2)=0
S.get_att_dc(2)=0
--------
S.get_synthesis_scale(3)=1
S.get_att_uc(3)=0
S.get_att_dc(3)=0
--------
S.get_synthesis_scale(4)=1
S.get_att_uc(4)=0
S.get_att_dc(4)=0
--------
S.get_synthesis_scale(5)=1
S.get_att_uc(5)=0
S.get_att_dc(5)=0
--------
S.get_synthesis_scale(6)=1
S.get_att_uc(6)=0
S.get_att_dc(6)=0
--------
S.get_synthesis_scale(7)=1
S.get_att_uc(7)=0
S.get_att_dc(7)=0
--------


Ok so with no attenuation and synthesis scale = 1, I see the following power levels in the signal analyzer near these frequencies;

4.25 GHz : -49 dBm

4.75 GHz : -51 dBm

5.25 GHz : -55 dBm

5.75 GHz : -56 dBm

6.25 GHz : -36 dBm

6.75 GHz : -39 dBm

7.25 GHz : -44 dBm

7.75 GHz : -49 dBm

and the HB noise floor looks kind of weird and wavy.  Let's kick the HB synthesisScale down to 0x2 and see if that helps match.

In [15]:
for band in range(4,8):
    S.set_synthesis_scale(band,0)

Dialing down the synthesis scale makes the waviness go away.  Now for HB we have ; 

6.25 GHz : -42 dBm

6.75 GHz : -45 dBm

7.25 GHz : -50 dBm

7.75 GHz : -55 dBm

Ok so let's try and get the power levels a little better by adjusting attenuators and tone powers.  HB bands 4 and 5 are still like 6dB higher than others.  Let's drop their amplitudes down by 3x.

In [21]:
for band in [4,5]:
    new_asa = np.zeros_like(S.get_amplitude_scale_array(band))
    new_asa[np.where(S.get_amplitude_scale_array(band)==12)]=10
    S.set_amplitude_scale_array(band,new_asa)

That helped a lot with the weird structure I was seeing.  Presumably we were hitting the mixers with too much power?

In [38]:
S.set_att_uc(0,7)
S.set_att_uc(1,4)
S.set_att_uc(2,0)
S.set_att_uc(3,0)

S.set_att_uc(4,10)
S.set_att_uc(5,8)
S.set_att_uc(6,6)

In [39]:
S.all_off()

[ 2022-02-14 09:38:26 ]  Turning off tones
[ 2022-02-14 09:38:53 ]  Turning off flux ramp
[ 2022-02-14 09:38:53 ]  Turning off all TES biases


## 2/14/22 Trying with SMuRF HB output connected directly to SA input

The system has been up for a while (~few days).  Is JESD still locked?

In [2]:
S.check_jesd(0)

[ 2022-02-15 06:29:18 ]  JESD Tx Okay
[ 2022-02-15 06:29:18 ]  JESD Rx Okay
[ 2022-02-15 06:29:19 ]  JESD health check finished after 1 seconds. The final status was Locked.


(True, True, 'Locked')

Yes.  Cyndia attached output of HB AMC directly into SA input.  Can we see RF noise in tone sidebands in this configuration?

In [43]:
S.all_off()
S.set_fixed_tone(4250,12)

[ 2022-02-15 05:45:23 ]  Turning off tones
[ 2022-02-15 05:45:46 ]  Turning off flux ramp
[ 2022-02-15 05:45:46 ]  Turning off all TES biases


(0, 0)

Yes, can see the noise in the tone sidebands.

Ok so first let's remake the wideband plot, but let's level it.  First let's gather some info.  I see some spurs with all tones off.  I see them at; 

4.00133 GHz : -39.39 dBm

4.249124 GHz : -34.45 dBm

4.49745 GHz : -57.08 dBm

4.91535 GHz : -44.69 dBm

5.50125 GHz : -30.95 dBm

5.99895 GHz : -23.06 dBm

6.14385 GHz : -39.18 dBm

6.49875 GHz : -26.54 dBm

6.14385 GHz : -39.43 dBm

6.49875 GHz : -26.59 dBm

6.75810 GHz : -50.12 dBm

7.37288 GHz : -52.88 dBm

7.49888 GHz : -51.75 dBm

7.99868 GHz : -30.36 dBm

some of those are LOs, others I'm not so sure.

What's the current settings?

In [3]:
for band in range(8):
    print(f'S.get_synthesis_scale({band})={S.get_synthesis_scale(band)}')
    print(f'S.get_att_uc({band})={S.get_att_uc(band)}')
    print(f'S.get_att_dc({band})={S.get_att_dc(band)}')
    print('--------')

S.get_synthesis_scale(0)=1
S.get_att_uc(0)=0
S.get_att_dc(0)=0
--------
S.get_synthesis_scale(1)=1
S.get_att_uc(1)=0
S.get_att_dc(1)=0
--------
S.get_synthesis_scale(2)=1
S.get_att_uc(2)=0
S.get_att_dc(2)=0
--------
S.get_synthesis_scale(3)=1
S.get_att_uc(3)=0
S.get_att_dc(3)=0
--------
S.get_synthesis_scale(4)=1
S.get_att_uc(4)=0
S.get_att_dc(4)=0
--------
S.get_synthesis_scale(5)=1
S.get_att_uc(5)=0
S.get_att_dc(5)=0
--------
S.get_synthesis_scale(6)=1
S.get_att_uc(6)=0
S.get_att_dc(6)=0
--------
S.get_synthesis_scale(7)=1
S.get_att_uc(7)=0
S.get_att_dc(7)=0
--------


Ok, set HB bands 4-8 to synthesisScale=0x0;

In [4]:
for band in range(4,8):
    S.set_synthesis_scale(band,0)

Fill the bands, restricting to 250 tones per 500 MHz band:

In [5]:
for band in range(8):
    sys.argv[1]=band
    exec(open('/usr/local/src/pysmurf/scratch/shawn/fill_band.py').read())

filling bands 0
-> Restricting nchan to 250.
filling bands 1
-> Restricting nchan to 250.
filling bands 2
-> Restricting nchan to 250.
filling bands 3
-> Restricting nchan to 250.
filling bands 4
-> Restricting nchan to 250.
filling bands 5
-> Restricting nchan to 250.
filling bands 6
-> Restricting nchan to 250.
filling bands 7
-> Restricting nchan to 250.


Lower amplitude of tones in bands 4 and 5 because those bands look a little overdriven

In [6]:
for band in [4,5]:
    new_asa = np.zeros_like(S.get_amplitude_scale_array(band))
    new_asa[np.where(S.get_amplitude_scale_array(band)==12)]=10
    S.set_amplitude_scale_array(band,new_asa)

Signal analyzer settings are span 4.2GHz, center frequency 6 GHz, Res BW 10kHz. 

In [7]:
for band in range(8):
    print(f'S.get_synthesis_scale({band})={S.get_synthesis_scale(band)}')
    print(f'S.get_att_uc({band})={S.get_att_uc(band)}')
    print(f'S.get_att_dc({band})={S.get_att_dc(band)}')
    print('--------')

S.get_synthesis_scale(0)=1
S.get_att_uc(0)=0
S.get_att_dc(0)=0
--------
S.get_synthesis_scale(1)=1
S.get_att_uc(1)=0
S.get_att_dc(1)=0
--------
S.get_synthesis_scale(2)=1
S.get_att_uc(2)=0
S.get_att_dc(2)=0
--------
S.get_synthesis_scale(3)=1
S.get_att_uc(3)=0
S.get_att_dc(3)=0
--------
S.get_synthesis_scale(4)=0
S.get_att_uc(4)=0
S.get_att_dc(4)=0
--------
S.get_synthesis_scale(5)=0
S.get_att_uc(5)=0
S.get_att_dc(5)=0
--------
S.get_synthesis_scale(6)=0
S.get_att_uc(6)=0
S.get_att_dc(6)=0
--------
S.get_synthesis_scale(7)=0
S.get_att_uc(7)=0
S.get_att_dc(7)=0
--------


Level 500 MHz bands using UC att

In [8]:
S.set_att_uc(0,11)
S.set_att_uc(1,5)
S.set_att_uc(2,2)
S.set_att_uc(3,2)

S.set_att_uc(4,12)
S.set_att_uc(5,8)
S.set_att_uc(6,10)
S.set_att_uc(7,0)

There's two channels in band 0 right on top of each other.  Turn one of them off.

In [11]:
S.which_on(0)

array([  3,   7,  11,  12,  13,  19,  22,  23,  28,  29,  31,  32,  34,
        35,  39,  40,  45,  47,  48,  50,  51,  52,  53,  55,  57,  58,
        60,  61,  66,  67,  68,  70,  71,  74,  75,  76,  77,  79,  82,
        84,  85,  87,  88,  89,  92,  95,  96,  98,  99, 100, 101, 103,
       104, 106, 107, 109, 111, 112, 114, 115, 116, 117, 122, 124, 125,
       127, 128, 132, 133, 134, 136, 141, 143, 144, 148, 149, 151, 152,
       153, 154, 156, 157, 159, 160, 165, 172, 173, 176, 178, 180, 181,
       183, 184, 185, 188, 189, 196, 200, 204, 205, 208, 210, 212, 213,
       215, 216, 224, 227, 228, 230, 232, 236, 237, 239, 240, 242, 243,
       244, 248, 249, 250, 251, 252, 253, 255, 258, 260, 261, 262, 266,
       267, 271, 274, 275, 276, 277, 279, 280, 282, 284, 287, 290, 293,
       294, 296, 306, 307, 309, 311, 312, 314, 315, 316, 322, 323, 325,
       326, 327, 330, 331, 332, 333, 338, 340, 341, 345, 347, 348, 352,
       354, 355, 356, 358, 359, 362, 363, 365, 367, 370, 371, 37

In [15]:
for chan in S.which_on(0):
    freq=S.channel_to_freq(0,chan)
    if freq>4248 and freq<4250:
        print(f'ch{chan} : {S.channel_to_freq(0,chan)}')

ch511 : 4248.946057796478


In [16]:
S.set_amplitude_scale_channel(0,511,0)

In [17]:
S.get_att_uc(0)

11

In [36]:
S.set_fixed_tone(4750,12)

(1, 0)

In [37]:
S.set_att_uc(1,0)

In [31]:
S.all_off()

[ 2022-02-15 07:52:45 ]  Turning off tones
[ 2022-02-15 07:53:08 ]  Turning off flux ramp
[ 2022-02-15 07:53:08 ]  Turning off all TES biases


In [35]:
S.check_jesd(0)

[ 2022-02-15 07:56:21 ]  JESD Tx Okay
[ 2022-02-15 07:56:21 ]  JESD Rx Okay
[ 2022-02-15 07:56:23 ]  JESD health check finished after 1 seconds. The final status was Locked.


(True, True, 'Locked')

In [51]:
S.all_off()

[ 2022-02-15 08:15:13 ]  Turning off tones
[ 2022-02-15 08:15:32 ]  Turning off flux ramp
[ 2022-02-15 08:15:32 ]  Turning off all TES biases


In [52]:
S.set_fixed_tone(4333,13)

(0, 324)

In [46]:
S.which_on(0)

array([], dtype=int64)